<a href="https://colab.research.google.com/github/AllysonAbreu/dw_contas_publicas_2022/blob/master/dea_despesas_dados_novos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports e instalações

In [26]:
# instalações das bibliotecas
!pip install basedosdados

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import datetime as dt
import basedosdados as bd
from google.colab import auth
from google.cloud import bigquery

# Imports dos dados

## Login no gcloud

In [28]:
auth.authenticate_user()

project = 'dados-ibge-384913'
location = 'US'
client = bigquery.Client(project=project, location=location)

## Querys para buscar no firebase

- Os dataframes com as tabelas utilizadas são de responsabilidade do projeto Base dos dados. Clique [aqui](https://basedosdados.org/) para conhecer mais.

In [29]:
query_saude = "select * from `basedosdados.br_ieps_saude.municipio` as db where db.id_municipio = '2503704'"
query_educacao = "select db.ano,db.atu_ei, db.atu_ei_creche, db.atu_ei_pre_escola,db.atu_ef, db.atu_ef_anos_iniciais,db.atu_ef_anos_finais,db.tdi_ef_anos_iniciais,db.tdi_ef_anos_finais,db.taxa_aprovacao_ef,db.taxa_aprovacao_ef_anos_iniciais,db.taxa_aprovacao_ef_anos_finais,db.taxa_reprovacao_ef,db.taxa_reprovacao_ef_anos_iniciais,db.taxa_reprovacao_ef_anos_finais,db.taxa_abandono_ef,db.taxa_abandono_ef_anos_iniciais,db.taxa_abandono_ef_anos_finais,db.dsu_ei,db.dsu_ei_creche,db.dsu_ei_pre_escola,db.dsu_ef,db.dsu_ef_anos_iniciais,db.dsu_ef_anos_finais from `basedosdados.br_inep_indicadores_educacionais.municipio` as db where db.id_municipio = '2503704'"

## Import dos dados do IBGE

### Indicadores saúde

In [30]:
# Para carregar os dados do firebase direto no pandas
df_ibge_saude = client.query(query_saude).to_dataframe()
df_ibge_saude.head(1)

,ano,id_municipio,sigla_uf,nome,cob_ab,cob_acs,cob_esf,cob_vac_bcg,cob_vac_rota,cob_vac_menin,...,desp_recp_saude_pc_mun,pct_desp_recp_saude_uf,desp_tot_saude_pc_uf,desp_recp_saude_pc_uf,desp_tot_saude_pc_mun_def,desp_recp_saude_pc_mun_def,desp_tot_saude_pc_uf_def,desp_recp_saude_pc_uf_def,num_familias_bf,gasto_pbf_pc_def
0,2010,2503704,PB,Cajazeiras,89.42,100.0,89.42,100.0,92.518703,0.0,...,86.08,NaN,NaN,NaN,562.799802,164.843332,NaN,NaN,NaN,286.193802


In [31]:
df_ibge_saude.dtypes

ano                             int64
id_municipio                   object
sigla_uf                       object
nome                           object
cob_ab                        float64
                               ...   
desp_recp_saude_pc_mun_def    float64
desp_tot_saude_pc_uf_def      float64
desp_recp_saude_pc_uf_def     float64
num_familias_bf               float64
gasto_pbf_pc_def              float64
Length: 68, dtype: object

### Indicadores educação básica

In [32]:
# Para carregar os dados do firebase direto no pandas
df_ibge_educacao = client.query(query_educacao).to_dataframe()
df_ibge_educacao.head(1)

,ano,atu_ei,atu_ei_creche,atu_ei_pre_escola,atu_ef,atu_ef_anos_iniciais,atu_ef_anos_finais,tdi_ef_anos_iniciais,tdi_ef_anos_finais,taxa_aprovacao_ef,...,taxa_reprovacao_ef_anos_finais,taxa_abandono_ef,taxa_abandono_ef_anos_iniciais,taxa_abandono_ef_anos_finais,dsu_ei,dsu_ei_creche,dsu_ei_pre_escola,dsu_ef,dsu_ef_anos_iniciais,dsu_ef_anos_finais
0,2011,NaN,NaN,NaN,16.2,NaN,NaN,39.2,NaN,86.3,...,NaN,10.5,10.5,NaN,NaN,NaN,NaN,16.7,NaN,16.7


In [33]:
df_ibge_educacao.dtypes

ano                                   int64
atu_ei                              float64
atu_ei_creche                       float64
atu_ei_pre_escola                   float64
atu_ef                              float64
atu_ef_anos_iniciais                float64
atu_ef_anos_finais                  float64
tdi_ef_anos_iniciais                float64
tdi_ef_anos_finais                  float64
taxa_aprovacao_ef                   float64
taxa_aprovacao_ef_anos_iniciais     float64
taxa_aprovacao_ef_anos_finais       float64
taxa_reprovacao_ef                  float64
taxa_reprovacao_ef_anos_iniciais    float64
taxa_reprovacao_ef_anos_finais      float64
taxa_abandono_ef                    float64
taxa_abandono_ef_anos_iniciais      float64
taxa_abandono_ef_anos_finais        float64
dsu_ei                              float64
dsu_ei_creche                       float64
dsu_ei_pre_escola                   float64
dsu_ef                              float64
dsu_ef_anos_iniciais            

### Ideb
- Os dados podem ser encontrados [aqui](https://www.gov.br/inep/pt-br/areas-de-atuacao/pesquisas-estatisticas-e-indicadores/ideb/resultados).

In [34]:
dic_dados_ideb_anos_iniciais = {2013:3.9,2015:4.0,2017:4.6,2019:5.3,2021:4.9} # 1 ao 5 ano
dic_dados_ideb_anos_finais = {2013:3.8,2015:4.0,2017:4.3,2019:4.6,2021:4.9} # 6 ao 9 ano
dic_dados_media_ideb = {2013:3.8,2015:4.0,2017:4.4,2019:4.9,2021:4.9} # média aritmética dos idebs

# transformando o dicionário em um dataframe pandas
df_dados_ideb = pd.DataFrame.from_dict(dic_dados_media_ideb, orient='index',columns=['media_ideb'])
df_dados_ideb.index.name='ano'
df_dados_ideb.reset_index(inplace=True)
df_dados_ideb

,ano,media_ideb
0,2013,3.8
1,2015,4.0
2,2017,4.4
3,2019,4.9
4,2021,4.9


### Média de Alunos por Turma
- Os dados podem ser encontrados [aqui](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/indicadores-educacionais/media-de-alunos-por-turma).

In [35]:
dic_alunos_por_turma = {2022:{'creche':23.1,'pre_escola':23.7,'ens_fund_anos_iniciais':23.5,'ens_fund_anos_finais':28.3,'ens_fund_total':25.5},
                        2021:{'creche':20.6,'pre_escola':23.4,'ens_fund_anos_iniciais':24.8,'ens_fund_anos_finais':29.2,'ens_fund_total':26.5},
                        2020:{'creche':20.1,'pre_escola':21.4,'ens_fund_anos_iniciais':23.9,'ens_fund_anos_finais':26.1,'ens_fund_total':24.6},
                        2019:{'creche':20.5,'pre_escola':18.5,'ens_fund_anos_iniciais':23.1,'ens_fund_anos_finais':26.6,'ens_fund_total':24.2},
                        2018:{'creche':17.6,'pre_escola':21.1,'ens_fund_anos_iniciais':22.7,'ens_fund_anos_finais':25.7,'ens_fund_total':23.7},
                        2017:{'creche':15.7,'pre_escola':19.7,'ens_fund_anos_iniciais':22.8,'ens_fund_anos_finais':25.2,'ens_fund_total':23.5},
                        2016:{'creche':16.1,'pre_escola':21.1,'ens_fund_anos_iniciais':22.2,'ens_fund_anos_finais':26.2,'ens_fund_total':23.6},
                        2015:{'creche':15.6,'pre_escola':21.4,'ens_fund_anos_iniciais':22.7,'ens_fund_anos_finais':22.6,'ens_fund_total':22.5},
                        2014:{'creche':19.9,'pre_escola':21.8,'ens_fund_anos_iniciais':22.7,'ens_fund_anos_finais':23.4,'ens_fund_total':22.6},
                        2013:{'creche':18.3,'pre_escola':19.3,'ens_fund_anos_iniciais':22.5,'ens_fund_anos_finais':23.8,'ens_fund_total':22.1}}

# criando dataframe
df_media_alunos_turma = pd.DataFrame(dic_alunos_por_turma).transpose()
df_media_alunos_turma.index.name='ano'
df_media_alunos_turma.reset_index(inplace=True)
df_media_alunos_turma

,ano,creche,pre_escola,ens_fund_anos_iniciais,ens_fund_anos_finais,ens_fund_total
0,2022,23.1,23.7,23.5,28.3,25.5
1,2021,20.6,23.4,24.8,29.2,26.5
2,2020,20.1,21.4,23.9,26.1,24.6
3,2019,20.5,18.5,23.1,26.6,24.2
4,2018,17.6,21.1,22.7,25.7,23.7
5,2017,15.7,19.7,22.8,25.2,23.5
6,2016,16.1,21.1,22.2,26.2,23.6
7,2015,15.6,21.4,22.7,22.6,22.5
8,2014,19.9,21.8,22.7,23.4,22.6
9,2013,18.3,19.3,22.5,23.8,22.1


### Porcentagem de Docentes com Curso Superior
- Os dados podem ser encontrados [aqui](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/indicadores-educacionais/percentual-de-docentes-com-curso-superior).

In [36]:
dic_porc_docentes_curso_superior = {2022:{'creche_%':88.6,'pre_escola_%':78,'ens_fund_anos_iniciais_%':82.2,'ens_fund_anos_finais_%':98,'ens_fund_total_%':91.4},
                                    2021:{'creche_%':97.1,'pre_escola_%':85.4,'ens_fund_anos_iniciais_%':88.5,'ens_fund_anos_finais_%':97.9,'ens_fund_total_%':94},
                                    2020:{'creche_%':94.9,'pre_escola_%':94.3,'ens_fund_anos_iniciais_%':85,'ens_fund_anos_finais_%':98.6,'ens_fund_total_%':92.7},
                                    2018:{'creche_%':100,'pre_escola_%':88.9,'ens_fund_anos_iniciais_%':90.2,'ens_fund_anos_finais_%':98.3,'ens_fund_total_%':94.5},
                                    2017:{'creche_%':90.0,'pre_escola_%':91.1,'ens_fund_anos_iniciais_%':90.5,'ens_fund_anos_finais_%':98.5,'ens_fund_total_%':94.8},
                                    2016:{'creche_%':82.6,'pre_escola_%':92.7,'ens_fund_anos_iniciais_%':95.8,'ens_fund_anos_finais_%':97.8,'ens_fund_total_%':95.8},
                                    2015:{'creche_%':77.8,'pre_escola_%':85,'ens_fund_anos_iniciais_%':88.8,'ens_fund_anos_finais_%':96.5,'ens_fund_total_%':93.1},
                                    2014:{'creche_%':78.6,'pre_escola_%':72.7,'ens_fund_anos_iniciais_%':83.8,'ens_fund_anos_finais_%':89.7,'ens_fund_total_%':87.1},
                                    2013:{'creche_%':57.1,'pre_escola_%':69.6,'ens_fund_anos_iniciais_%':80.9,'ens_fund_anos_finais_%':87.4,'ens_fund_total_%':84.6}}

# criando dataframe
df_porc_docentes_curso_superior = pd.DataFrame(dic_porc_docentes_curso_superior).transpose()
df_porc_docentes_curso_superior.index.name='ano'
df_porc_docentes_curso_superior.reset_index(inplace=True)
df_porc_docentes_curso_superior

,ano,creche_%,pre_escola_%,ens_fund_anos_iniciais_%,ens_fund_anos_finais_%,ens_fund_total_%
0,2022,88.6,78.0,82.2,98.0,91.4
1,2021,97.1,85.4,88.5,97.9,94.0
2,2020,94.9,94.3,85.0,98.6,92.7
3,2018,100.0,88.9,90.2,98.3,94.5
4,2017,90.0,91.1,90.5,98.5,94.8
5,2016,82.6,92.7,95.8,97.8,95.8
6,2015,77.8,85.0,88.8,96.5,93.1
7,2014,78.6,72.7,83.8,89.7,87.1
8,2013,57.1,69.6,80.9,87.4,84.6


### Taxa de Distorção Idade-Série
- Os dados podem ser encontrados [aqui](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/indicadores-educacionais/taxas-de-distorcao-idade-serie).

In [37]:
dic_taxa_distocao_idade_serie = {2022:{'ens_fund_anos_iniciais_%':8.8,'ens_fund_anos_finais_%':33.9,'ens_fund_total_%':20.4},
                                    2021:{'ens_fund_anos_iniciais_%':14.4,'ens_fund_anos_finais_%':37.9,'ens_fund_total_%':25.0},
                                    2020:{'ens_fund_anos_iniciais_%':20.7,'ens_fund_anos_finais_%':40.9,'ens_fund_total_%':29.1},
                                    2019:{'ens_fund_anos_iniciais_%':23.7,'ens_fund_anos_finais_%':42.2,'ens_fund_total_%':31.4},
                                    2018:{'ens_fund_anos_iniciais_%':24.9,'ens_fund_anos_finais_%':44.0,'ens_fund_total_%':32.1},
                                    2017:{'ens_fund_anos_iniciais_%':26.2,'ens_fund_anos_finais_%':46.8,'ens_fund_total_%':34.0},
                                    2016:{'ens_fund_anos_iniciais_%':26.4,'ens_fund_anos_finais_%':45.7,'ens_fund_total_%':33.9},
                                    2015:{'ens_fund_anos_iniciais_%':27.6,'ens_fund_anos_finais_%':49.8,'ens_fund_total_%':36.0},
                                    2014:{'ens_fund_anos_iniciais_%':29.4,'ens_fund_anos_finais_%':51.5,'ens_fund_total_%':38.3},
                                    2013:{'ens_fund_anos_iniciais_%':30.3,'ens_fund_anos_finais_%':51.5,'ens_fund_total_%':39.0}}

# criando dataframe
df_taxa_distocao_idade_serie = pd.DataFrame(dic_taxa_distocao_idade_serie).transpose()
df_taxa_distocao_idade_serie.index.name='ano'
df_taxa_distocao_idade_serie.reset_index(inplace=True)
df_taxa_distocao_idade_serie

,ano,ens_fund_anos_iniciais_%,ens_fund_anos_finais_%,ens_fund_total_%
0,2022,8.8,33.9,20.4
1,2021,14.4,37.9,25.0
2,2020,20.7,40.9,29.1
3,2019,23.7,42.2,31.4
4,2018,24.9,44.0,32.1
5,2017,26.2,46.8,34.0
6,2016,26.4,45.7,33.9
7,2015,27.6,49.8,36.0
8,2014,29.4,51.5,38.3
9,2013,30.3,51.5,39.0


### Remuneração Média dos Docentes
- Os dados podem ser encontrados [aqui](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/indicadores-educacionais/remuneracao-media-dos-docentes).

In [38]:
dic_remuneracao_media_docentes = {2018:{'ens_superior': 258, 'media_ens_superior_R$':4390.05, 'sem_ens_superior':15, 'media_sem_ens_superior_R$':3805.75,'total':273,'carga_horaria_semanal_H':40,'remuneracao_media_total_R$':4359.60},
                                  2017:{'ens_superior': 258, 'media_ens_superior_R$':4321.36, 'sem_ens_superior':20, 'media_sem_ens_superior_R$':3456.23,'total':278,'carga_horaria_semanal_H':40,'remuneracao_media_total_R$':4273.53},
                                  2016:{'ens_superior': 252, 'media_ens_superior_R$':3858.64, 'sem_ens_superior':17, 'media_sem_ens_superior_R$':3271.40,'total':269,'carga_horaria_semanal_H':40,'remuneracao_media_total_R$':3823.97},
                                  2015:{'ens_superior': 255, 'media_ens_superior_R$':3382.48, 'sem_ens_superior':27, 'media_sem_ens_superior_R$':2171.45,'total':282,'carga_horaria_semanal_H':40,'remuneracao_media_total_R$':3287.65},
                                  2014:{'ens_superior': 252, 'media_ens_superior_R$':2832.46, 'sem_ens_superior':49, 'media_sem_ens_superior_R$':1292.05,'total':301,'carga_horaria_semanal_H':40,'remuneracao_media_total_R$':2832.46}}

# criando dataframe
df_remuneracao_media_docentes = pd.DataFrame(dic_remuneracao_media_docentes).transpose()
df_remuneracao_media_docentes.index.name='ano'
df_remuneracao_media_docentes.reset_index(inplace=True)
df_remuneracao_media_docentes

,ano,ens_superior,media_ens_superior_R$,sem_ens_superior,media_sem_ens_superior_R$,total,carga_horaria_semanal_H,remuneracao_media_total_R$
0,2018,258.0,4390.05,15.0,3805.75,273.0,40.0,4359.60
1,2017,258.0,4321.36,20.0,3456.23,278.0,40.0,4273.53
2,2016,252.0,3858.64,17.0,3271.40,269.0,40.0,3823.97
3,2015,255.0,3382.48,27.0,2171.45,282.0,40.0,3287.65
4,2014,252.0,2832.46,49.0,1292.05,301.0,40.0,2832.46


### Taxa de Rendimento
- Os dados podem ser encontrados [aqui](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/indicadores-educacionais/taxas-de-rendimento).

In [39]:
dic_taxa_rendimento = {2021:{'aprovacao_anos_iniciais_ens_fundamental_%': 99.5,'aprovacao_anos_finais_ens_fundamental_%':97.9,'total_aprovacao_ens_fundamental_%':98.8,'reprovacao_anos_iniciais_ens_fundamental_%':0.0,'reprovacao_anos_finais_ens_fundamental_%':0.2,'total_reprovacao_ens_fundamental_%':0.1,'abandono_anos_iniciais_ens_fundamental_%':0.5,'abandono_anos_finais_ens_fundamental_%':1.9,'total_abando_ens_fundamental':1.1},
                      2020:{'aprovacao_anos_iniciais_ens_fundamental_%': 99.0,'aprovacao_anos_finais_ens_fundamental_%':97.4,'total_aprovacao_ens_fundamental_%':98.4,'reprovacao_anos_iniciais_ens_fundamental_%':0.0,'reprovacao_anos_finais_ens_fundamental_%':0.2,'total_reprovacao_ens_fundamental_%':0.1,'abandono_anos_iniciais_ens_fundamental_%':1.0,'abandono_anos_finais_ens_fundamental_%':2.4,'total_abando_ens_fundamental':1.5},
                      2019:{'aprovacao_anos_iniciais_ens_fundamental_%': 91.5,'aprovacao_anos_finais_ens_fundamental_%':80.2,'total_aprovacao_ens_fundamental_%':86.8,'reprovacao_anos_iniciais_ens_fundamental_%':6.3,'reprovacao_anos_finais_ens_fundamental_%':14.1,'total_reprovacao_ens_fundamental_%':9.5,'abandono_anos_iniciais_ens_fundamental_%':2.2,'abandono_anos_finais_ens_fundamental_%':5.7,'total_abando_ens_fundamental':3.7},
                      2018:{'aprovacao_anos_iniciais_ens_fundamental_%': 82.7,'aprovacao_anos_finais_ens_fundamental_%':76.4,'total_aprovacao_ens_fundamental_%':80.3,'reprovacao_anos_iniciais_ens_fundamental_%':14.8,'reprovacao_anos_finais_ens_fundamental_%':16.6,'total_reprovacao_ens_fundamental_%':15.5,'abandono_anos_iniciais_ens_fundamental_%':2.5,'abandono_anos_finais_ens_fundamental_%':7.0,'total_abando_ens_fundamental':4.2},
                      2017:{'aprovacao_anos_iniciais_ens_fundamental_%': 82.0,'aprovacao_anos_finais_ens_fundamental_%':72.7,'total_aprovacao_ens_fundamental_%':78.5,'reprovacao_anos_iniciais_ens_fundamental_%':14.3,'reprovacao_anos_finais_ens_fundamental_%':19.6,'total_reprovacao_ens_fundamental_%':16.3,'abandono_anos_iniciais_ens_fundamental_%':3.7,'abandono_anos_finais_ens_fundamental_%':7.7,'total_abando_ens_fundamental':5.2},
                      2016:{'aprovacao_anos_iniciais_ens_fundamental_%': 81.6,'aprovacao_anos_finais_ens_fundamental_%':70.7,'total_aprovacao_ens_fundamental_%':77.4,'reprovacao_anos_iniciais_ens_fundamental_%':14.8,'reprovacao_anos_finais_ens_fundamental_%':18.9,'total_reprovacao_ens_fundamental_%':16.3,'abandono_anos_iniciais_ens_fundamental_%':3.6,'abandono_anos_finais_ens_fundamental_%':10.4,'total_abando_ens_fundamental':6.3},
                      2015:{'aprovacao_anos_iniciais_ens_fundamental_%': 79.1,'aprovacao_anos_finais_ens_fundamental_%':69.7,'total_aprovacao_ens_fundamental_%':75.6,'reprovacao_anos_iniciais_ens_fundamental_%':14.8,'reprovacao_anos_finais_ens_fundamental_%':18.6,'total_reprovacao_ens_fundamental_%':16.2,'abandono_anos_iniciais_ens_fundamental_%':6.1,'abandono_anos_finais_ens_fundamental_%':11.7,'total_abando_ens_fundamental':8.2},
                      2014:{'aprovacao_anos_iniciais_ens_fundamental_%': 81.1,'aprovacao_anos_finais_ens_fundamental_%':69.0,'total_aprovacao_ens_fundamental_%':76.2,'reprovacao_anos_iniciais_ens_fundamental_%':14.8,'reprovacao_anos_finais_ens_fundamental_%':16.8,'total_reprovacao_ens_fundamental_%':15.6,'abandono_anos_iniciais_ens_fundamental_%':4.1,'abandono_anos_finais_ens_fundamental_%':14.2,'total_abando_ens_fundamental':8.2},
                      2013:{'aprovacao_anos_iniciais_ens_fundamental_%': 84.5,'aprovacao_anos_finais_ens_fundamental_%':67.2,'total_aprovacao_ens_fundamental_%':77.4,'reprovacao_anos_iniciais_ens_fundamental_%':12.2,'reprovacao_anos_finais_ens_fundamental_%':20.4,'total_reprovacao_ens_fundamental_%':15.6,'abandono_anos_iniciais_ens_fundamental_%':3.3,'abandono_anos_finais_ens_fundamental_%':12.4,'total_abando_ens_fundamental':7.0}}

# criando dataframe
df_taxa_rendimento = pd.DataFrame(dic_taxa_rendimento).transpose()
df_taxa_rendimento.index.name='ano'
df_taxa_rendimento.reset_index(inplace=True)
df_taxa_rendimento

,ano,aprovacao_anos_iniciais_ens_fundamental_%,aprovacao_anos_finais_ens_fundamental_%,total_aprovacao_ens_fundamental_%,reprovacao_anos_iniciais_ens_fundamental_%,reprovacao_anos_finais_ens_fundamental_%,total_reprovacao_ens_fundamental_%,abandono_anos_iniciais_ens_fundamental_%,abandono_anos_finais_ens_fundamental_%,total_abando_ens_fundamental
0,2021,99.5,97.9,98.8,0.0,0.2,0.1,0.5,1.9,1.1
1,2020,99.0,97.4,98.4,0.0,0.2,0.1,1.0,2.4,1.5
2,2019,91.5,80.2,86.8,6.3,14.1,9.5,2.2,5.7,3.7
3,2018,82.7,76.4,80.3,14.8,16.6,15.5,2.5,7.0,4.2
4,2017,82.0,72.7,78.5,14.3,19.6,16.3,3.7,7.7,5.2
5,2016,81.6,70.7,77.4,14.8,18.9,16.3,3.6,10.4,6.3
6,2015,79.1,69.7,75.6,14.8,18.6,16.2,6.1,11.7,8.2
7,2014,81.1,69.0,76.2,14.8,16.8,15.6,4.1,14.2,8.2
8,2013,84.5,67.2,77.4,12.2,20.4,15.6,3.3,12.4,7.0


## Import dos dados em .csv

In [40]:
2503704

2503704

In [41]:
df_despesas = pd.read_csv('dados_despesas.csv', sep=';')
df_despesas

,DATA_FATO,FONTE_RECURSO,ORGAO_INTERNO,ORGAO_VINCULADO,ELEMENTO,SUBELEMENTO,FUNCAO,SUBFUNCAO,NATUREZA,VALOR_FIXADO,VALOR_EMPENHADO,VALOR_LIQUIDADO,VALOR_PAGO,SALDO,CODIGO
0,2013-12-31,recursos ordinarios,secretaria municipal de infra estrutura,prefeitura municipal,39,2044,4,122,3,2911694.0,5120.00,5120.0,0.0,5120.00,80e7979ff78e14
1,2013-12-31,recursos ordinarios,secretaria municipal de educacao,prefeitura municipal,36,2029,13,392,3,107000.0,6950.00,6950.0,0.0,6950.00,4be681d5c1e352
2,2013-12-31,recursos ordinarios,secretaria municipal de educacao,prefeitura municipal,36,2029,13,392,3,107000.0,800.00,800.0,0.0,800.00,8ff22c743e712e
3,2013-12-31,recursos ordinarios,secretaria municipal de educacao,prefeitura municipal,36,2029,13,392,3,107000.0,400.00,400.0,0.0,400.00,1db102414906f0
4,2013-12-31,recursos ordinarios,secretaria municipal de educacao,prefeitura municipal,36,2029,13,392,3,107000.0,600.00,600.0,0.0,600.00,5125ea1b50db6d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86836,2022-12-30,sem informacoes,fundo municipal de saude,fundo municipal de saude,339030,101,1,1,10,0.0,25673.02,0.0,0.0,25673.02,7bfd3e9d8aa27b
86837,2022-12-30,sem informacoes,fundo municipal de saude,fundo municipal de saude,339030,101,1,1,10,0.0,23333.43,0.0,0.0,23333.43,7bfd3e9d8aa27b
86838,2022-12-30,sem informacoes,fundo municipal de saude,fundo municipal de saude,339030,101,1,1,10,0.0,17920.75,0.0,0.0,17920.75,7bfd3e9d8aa27b
86839,2022-12-30,sem informacoes,fundo municipal de saude,fundo municipal de saude,339030,101,1,1,10,0.0,3842.60,0.0,0.0,3842.60,7bfd3e9d8aa27b


In [42]:
df_despesas.dtypes

DATA_FATO           object
FONTE_RECURSO       object
ORGAO_INTERNO       object
ORGAO_VINCULADO     object
ELEMENTO             int64
SUBELEMENTO          int64
FUNCAO               int64
SUBFUNCAO            int64
NATUREZA             int64
VALOR_FIXADO       float64
VALOR_EMPENHADO    float64
VALOR_LIQUIDADO    float64
VALOR_PAGO         float64
SALDO              float64
CODIGO              object
dtype: object

## Conversões nos dados

### Conversão e tratamento dos dados nas colunas do DF df_ibge_saude

In [43]:
# dados antes da conversão
df_ibge_saude.dtypes

ano                             int64
id_municipio                   object
sigla_uf                       object
nome                           object
cob_ab                        float64
                               ...   
desp_recp_saude_pc_mun_def    float64
desp_tot_saude_pc_uf_def      float64
desp_recp_saude_pc_uf_def     float64
num_familias_bf               float64
gasto_pbf_pc_def              float64
Length: 68, dtype: object

In [44]:
# dados da coluna ano após a conversão
df_ibge_saude['ano'] = pd.to_datetime(df_ibge_saude['ano'], format='%Y')
df_ibge_saude.dtypes

ano                           datetime64[ns]
id_municipio                          object
sigla_uf                              object
nome                                  object
cob_ab                               float64
                                   ...      
desp_recp_saude_pc_mun_def           float64
desp_tot_saude_pc_uf_def             float64
desp_recp_saude_pc_uf_def            float64
num_familias_bf                      float64
gasto_pbf_pc_def                     float64
Length: 68, dtype: object

### Conversão e tratamento dos dados da coluna tempo do DF dados_despesas

In [45]:
# dados antes da conversão
df_despesas.dtypes

DATA_FATO           object
FONTE_RECURSO       object
ORGAO_INTERNO       object
ORGAO_VINCULADO     object
ELEMENTO             int64
SUBELEMENTO          int64
FUNCAO               int64
SUBFUNCAO            int64
NATUREZA             int64
VALOR_FIXADO       float64
VALOR_EMPENHADO    float64
VALOR_LIQUIDADO    float64
VALOR_PAGO         float64
SALDO              float64
CODIGO              object
dtype: object

In [46]:
# dados após a conversão
df_despesas['DATA_FATO'] = pd.to_datetime(df_despesas['DATA_FATO'])
df_despesas.dtypes

DATA_FATO          datetime64[ns]
FONTE_RECURSO              object
ORGAO_INTERNO              object
ORGAO_VINCULADO            object
ELEMENTO                    int64
SUBELEMENTO                 int64
FUNCAO                      int64
SUBFUNCAO                   int64
NATUREZA                    int64
VALOR_FIXADO              float64
VALOR_EMPENHADO           float64
VALOR_LIQUIDADO           float64
VALOR_PAGO                float64
SALDO                     float64
CODIGO                     object
dtype: object

### Conversão na coluna ano do DF df_ibge_educacao

In [47]:
# dados originais
df_ibge_educacao.dtypes

ano                                   int64
atu_ei                              float64
atu_ei_creche                       float64
atu_ei_pre_escola                   float64
atu_ef                              float64
atu_ef_anos_iniciais                float64
atu_ef_anos_finais                  float64
tdi_ef_anos_iniciais                float64
tdi_ef_anos_finais                  float64
taxa_aprovacao_ef                   float64
taxa_aprovacao_ef_anos_iniciais     float64
taxa_aprovacao_ef_anos_finais       float64
taxa_reprovacao_ef                  float64
taxa_reprovacao_ef_anos_iniciais    float64
taxa_reprovacao_ef_anos_finais      float64
taxa_abandono_ef                    float64
taxa_abandono_ef_anos_iniciais      float64
taxa_abandono_ef_anos_finais        float64
dsu_ei                              float64
dsu_ei_creche                       float64
dsu_ei_pre_escola                   float64
dsu_ef                              float64
dsu_ef_anos_iniciais            

In [48]:
# dados apos conversao para dt
df_ibge_educacao['ano'] = pd.to_datetime(df_ibge_educacao['ano'], format='%Y')
df_ibge_educacao.head(1)

,ano,atu_ei,atu_ei_creche,atu_ei_pre_escola,atu_ef,atu_ef_anos_iniciais,atu_ef_anos_finais,tdi_ef_anos_iniciais,tdi_ef_anos_finais,taxa_aprovacao_ef,...,taxa_reprovacao_ef_anos_finais,taxa_abandono_ef,taxa_abandono_ef_anos_iniciais,taxa_abandono_ef_anos_finais,dsu_ei,dsu_ei_creche,dsu_ei_pre_escola,dsu_ef,dsu_ef_anos_iniciais,dsu_ef_anos_finais
0,2011-01-01,NaN,NaN,NaN,16.2,NaN,NaN,39.2,NaN,86.3,...,NaN,10.5,10.5,NaN,NaN,NaN,NaN,16.7,NaN,16.7


In [49]:
df_ibge_educacao.fillna(0.0,inplace=True)

In [50]:
# dados apos a conversao
df_ibge_educacao.dtypes

ano                                 datetime64[ns]
atu_ei                                     float64
atu_ei_creche                              float64
atu_ei_pre_escola                          float64
atu_ef                                     float64
atu_ef_anos_iniciais                       float64
atu_ef_anos_finais                         float64
tdi_ef_anos_iniciais                       float64
tdi_ef_anos_finais                         float64
taxa_aprovacao_ef                          float64
taxa_aprovacao_ef_anos_iniciais            float64
taxa_aprovacao_ef_anos_finais              float64
taxa_reprovacao_ef                         float64
taxa_reprovacao_ef_anos_iniciais           float64
taxa_reprovacao_ef_anos_finais             float64
taxa_abandono_ef                           float64
taxa_abandono_ef_anos_iniciais             float64
taxa_abandono_ef_anos_finais               float64
dsu_ei                                     float64
dsu_ei_creche                  

# Análises

## Séries históricas

### Time series da população cajazeirense entre 1991 e 2021

In [51]:
df_ibge_populacao_index_ano = df_ibge_populacao.set_index('ano')

NameError: ignored

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_ibge_populacao_index_ano.index, df_ibge_populacao_index_ano['populacao'], label='Total Anual')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('População Cajazeirense entre 1991 e 2021 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
df_ibge_populacao_index_ano['variacao_anual'] = df_ibge_populacao_index_ano['populacao'].diff()
df_ibge_populacao_index_ano['porcentagem_variacao_anual_%'] = ((df_ibge_populacao_index_ano['variacao_anual'] / df_ibge_populacao_index_ano['populacao'].shift(1)) * 100).round(2)
df_ibge_populacao_index_ano['aceleracao_variacao_anual'] = df_ibge_populacao_index_ano['variacao_anual'].diff()
df_ibge_populacao_index_ano['porcentagem_aceleracao_variacao_anual_%'] = ((df_ibge_populacao_index_ano['aceleracao_variacao_anual'] / df_ibge_populacao_index_ano['variacao_anual'].shift(1)) * 100).round(2)
df_ibge_populacao_index_ano

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_ibge_populacao_index_ano.index, df_ibge_populacao_index_ano['variacao_anual'], label='Variação')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Variação Anual Populacional Cajazeirense entre 1991 e 2021 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_ibge_populacao_index_ano.index, df_ibge_populacao_index_ano['aceleracao_variacao_anual'], label='Aceleração')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Aceleração Crescimento Anual Populacional Cajazeirense entre 1991 e 2021 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
print(f"Total da variação populacional entre 1991 e 2021: {df_ibge_populacao_index_ano['variacao_anual'].sum()}")
print(f"Média móvel da variação anual: {(df_ibge_populacao_index_ano['variacao_anual'].sum()/df_ibge_populacao_index_ano.shape[0]).round(2)}")

In [ ]:
print(f"Total da aceleração da variação populacional entre 1991 e 2021: {df_ibge_populacao_index_ano['aceleracao_variacao_anual'].sum()}")
print(f"Média móvel anual da aceleração da variação populacional entre 1991 e 2021: {(df_ibge_populacao_index_ano['aceleracao_variacao_anual'].sum()/df_ibge_populacao_index_ano.shape[0]).round(2)}")

In [ ]:
print(f"Porcentagem da variação populacional entre 1991 e 2021: {df_ibge_populacao_index_ano['porcentagem_variacao_anual_%'].sum().round(2)}%")
print(f"Média percentual móvel anual da variação populacional entre 1991 e 2021: {(df_ibge_populacao_index_ano['porcentagem_variacao_anual_%'].sum()/df_ibge_populacao_index_ano.shape[0]).round(2)}%")

In [ ]:
print(f"Porcentagem da aceleração da variação populacional entre 1991 e 2021: {df_ibge_populacao_index_ano['porcentagem_aceleracao_variacao_anual_%'].sum().round(2)}%")
print(f"Média percentual móvel anual da aceleração da variação populacional entre 1991 e 2021: {(df_ibge_populacao_index_ano['porcentagem_aceleracao_variacao_anual_%'].sum()/df_ibge_populacao_index_ano.shape[0]).round(2)}%")

### Time series do pib entre 1999 e 2020

In [ ]:
# verificando se há valores nulos
df_ibge_pib.isna().value_counts()

In [ ]:
# verificando o tipo das colunas
df_ibge_pib.dtypes

In [ ]:
# criando df para analises dos dados
df_valor_pib_analise = df_ibge_pib[['ano','pib', 'impostos_liquidos', 'va', 'va_agropecuaria', 'va_industria', 'va_servicos', 'va_adespss']]

df_valor_pib_analise.set_index('ano', inplace=True)
df_valor_pib_analise

#### PIB

In [ ]:
df_pib_analise = df_valor_pib_analise.copy()
df_pib_analise.drop(columns=['impostos_liquidos','va','va_agropecuaria','va_industria','va_servicos','va_adespss'], inplace=True)

pd.options.display.float_format = '{:.2f}'.format
print(df_pib_analise)

In [ ]:
# inferindo variação e acelaração anual do pib
df_pib_analise['variacao_anual'] = df_pib_analise['pib'].diff()
df_pib_analise['porcentagem_variacao_anual_%'] = ((df_pib_analise['variacao_anual'] / df_pib_analise['pib'].shift(1)) * 100).round(2)
df_pib_analise['aceleracao_variacao_anual'] = df_pib_analise['variacao_anual'].diff()
df_pib_analise['porcentagem_aceleracao_variacao_anual_%'] = ((df_pib_analise['aceleracao_variacao_anual'] / df_pib_analise['variacao_anual'].shift(1)) * 100).round(2)
df_pib_analise

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_pib_analise.index, df_pib_analise['pib'], label='Valor Anual - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('PIB municipal anual bruto entre os anos de 1999 a 2020')
ax.xaxis.label.set_visible(False)  # esconde a legenda do eixo X

# Configura o eixo x para não mostrar nenhum rótulo
# plt.gca().set_xticklabels([])

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_pib_analise.index, df_pib_analise['variacao_anual'], label='Valor Anual - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Variação do PIB municipal bruto entre os anos de 1999 a 2020')
ax.xaxis.label.set_visible(False)  # esconde a legenda do eixo X

# Configura o eixo x para não mostrar nenhum rótulo
# plt.gca().set_xticklabels([])

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_pib_analise.index, df_pib_analise['aceleracao_variacao_anual'], label='Valor Anual - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Aceleração do PIB municipal anual bruto entre os anos de 1999 a 2020')
ax.xaxis.label.set_visible(False)  # esconde a legenda do eixo X

# Configura o eixo x para não mostrar nenhum rótulo
# plt.gca().set_xticklabels([])

# Exibe o gráfico
plt.show()

In [ ]:
print(f"Total da variação do PIB municipal: R$ {df_pib_analise['variacao_anual'].sum().round(2)}")
print(f"Média móvel anual da variação do PIB municipal: R$ {(df_pib_analise['variacao_anual'].sum()/df_pib_analise.shape[0]).round(2)}")

In [ ]:
print(f"Total da aceleração da variação do PIB municipal: R$ {df_pib_analise['aceleracao_variacao_anual'].sum().round(2)}")
print(f"Média móvel anual da aceleração da variação do PIB municipal: R$ {(df_pib_analise['aceleracao_variacao_anual'].sum()/df_pib_analise.shape[0]).round(2)}")

In [ ]:
print(f"Porcentagem da variação do PIB municipal: {df_pib_analise['porcentagem_variacao_anual_%'].sum().round(2)}%")
print(f"Média móvel percentual anual da variação do PIB municipal: {(df_pib_analise['porcentagem_variacao_anual_%'].sum()/df_pib_analise.shape[0]).round(2)}%")

In [ ]:
print(f"Porcentagem da aceleração da variação do PIB municipal: {df_pib_analise['porcentagem_aceleracao_variacao_anual_%'].sum().round(2)}%")
print(f"Média móvel percentual anual da acelereção variação do PIB municipal: {(df_pib_analise['porcentagem_aceleracao_variacao_anual_%'].sum()/df_pib_analise.shape[0]).round(2)}%")

#### Valores brutos específicos

In [ ]:
df_va_analise = df_valor_pib_analise.copy()
df_va_analise.drop(columns=['pib', 'impostos_liquidos'], inplace=True)

pd.options.display.float_format = '{:.2f}'.format
print(df_va_analise)

In [ ]:
# inferindo variação e acelaração anual dos valores brutos
df_va_analise['variacao_anual_va'] = df_va_analise['va'].diff()
df_va_analise['porcentagem_variacao_anual_va_%'] = ((df_va_analise['variacao_anual_va'] / df_va_analise['va'].shift(1)) * 100).round(2)
df_va_analise['aceleracao_variacao_anual_va'] = df_va_analise['variacao_anual_va'].diff()
df_va_analise['porcentagem_aceleracao_variacao_anual_va_%'] = ((df_va_analise['aceleracao_variacao_anual_va'] / df_va_analise['variacao_anual_va'].shift(1)) * 100).round(2)

In [ ]:
# inferindo variação e acelaração anual dos valores brutos
df_va_analise['variacao_anual_va_agropecuaria'] = df_va_analise['va_agropecuaria'].diff()
df_va_analise['porcentagem_variacao_anual_va_agropecuaria_%'] = ((df_va_analise['variacao_anual_va_agropecuaria'] / df_va_analise['va_agropecuaria'].shift(1)) * 100).round(2)
df_va_analise['aceleracao_variacao_anual_va_agropecuaria'] = df_va_analise['variacao_anual_va_agropecuaria'].diff()
df_va_analise['porcentagem_aceleracao_variacao_anual_va_agropecuaria_%'] = ((df_va_analise['aceleracao_variacao_anual_va_agropecuaria'] / df_va_analise['variacao_anual_va_agropecuaria'].shift(1)) * 100).round(2)

In [ ]:
# inferindo variação e acelaração anual dos valores brutos
df_va_analise['variacao_anual_va_industria'] = df_va_analise['va_industria'].diff()
df_va_analise['porcentagem_variacao_anual_va_industria_%'] = ((df_va_analise['variacao_anual_va_industria'] / df_va_analise['va_industria'].shift(1)) * 100).round(2)
df_va_analise['aceleracao_variacao_anual_va_industria'] = df_va_analise['variacao_anual_va_industria'].diff()
df_va_analise['porcentagem_aceleracao_variacao_anual_va_industria_%'] = ((df_va_analise['aceleracao_variacao_anual_va_industria'] / df_va_analise['variacao_anual_va_industria'].shift(1)) * 100).round(2)

In [ ]:
# inferindo variação e acelaração anual dos valores brutos
df_va_analise['variacao_anual_va_servicos'] = df_va_analise['va_servicos'].diff()
df_va_analise['porcentagem_variacao_anual_va_servicos_%'] = ((df_va_analise['variacao_anual_va_servicos'] / df_va_analise['va_servicos'].shift(1)) * 100).round(2)
df_va_analise['aceleracao_variacao_anual_va_servicos'] = df_va_analise['variacao_anual_va_servicos'].diff()
df_va_analise['porcentagem_aceleracao_variacao_anual_va_servicos_%'] = ((df_va_analise['aceleracao_variacao_anual_va_servicos'] / df_va_analise['variacao_anual_va_servicos'].shift(1)) * 100).round(2)

In [ ]:
# inferindo variação e acelaração anual dos valores brutos
df_va_analise['variacao_anual_va_adespss'] = df_va_analise['va_adespss'].diff()
df_va_analise['porcentagem_variacao_anual_va_adespss_%'] = ((df_va_analise['variacao_anual_va_adespss'] / df_va_analise['va_adespss'].shift(1)) * 100).round(2)
df_va_analise['aceleracao_variacao_anual_va_adespss'] = df_va_analise['variacao_anual_va_adespss'].diff()
df_va_analise['porcentagem_aceleracao_variacao_anual_va_adespss_%'] = ((df_va_analise['aceleracao_variacao_anual_va_adespss'] / df_va_analise['variacao_anual_va_adespss'].shift(1)) * 100).round(2)
df_va_analise

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['va'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Valores adicionado bruto a preços correntes 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['variacao_anual_va'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Variação anual dos valores adicionado bruto a preços correntes 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['aceleracao_variacao_anual_va'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Aceleração da variação anual dos valores adicionado bruto a preços correntes 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['va_agropecuaria'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Valores adicionado bruto a preços correntes da agropecuária 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['variacao_anual_va_agropecuaria'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Variação anual dos valores adicionado bruto a preços correntes da agropecuária 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['aceleracao_variacao_anual_va_agropecuaria'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Aceleração da variação anual dos valores adicionado bruto a preços correntes da agropecuária 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['va_industria'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Valores adicionado bruto a preços correntes da indústria 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['variacao_anual_va_industria'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Variação anual dos valores adicionado bruto a preços correntes da indústria 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['aceleracao_variacao_anual_va_industria'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Aceleração da variação anual dos valores adicionado bruto a preços correntes da indústria 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['va_servicos'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Valores adicionado bruto a preços correntes dos serviços 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['variacao_anual_va_servicos'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Variação anual dos valores adicionado bruto a preços correntes dos serviços 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['aceleracao_variacao_anual_va_servicos'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Aceleração da variação anual dos valores adicionado bruto a preços correntes dos serviços 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['va_adespss'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Valores adicionado bruto a preços correntes dos serviços públicos 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['variacao_anual_va_adespss'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Variação anual dos valores adicionado bruto a preços correntes dos serviços públicos 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_va_analise.index, df_va_analise['aceleracao_variacao_anual_va_adespss'], label='Valor - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Aceleração da variação anual dos valores adicionado bruto a preços correntes dos serviços públicos 1990 e 2020 - IBGE')

# Exibe o gráfico
plt.show()

In [ ]:
# Variação da receita bruta anual
print(f"Total da variação anual da receita bruta anual entre 1999 e 2020: R$ {df_va_analise['variacao_anual_va'].sum().round(2)}")
print(f"Média móvel da variação anual: R$ {(df_va_analise['variacao_anual_va'].sum()/df_va_analise.shape[0]).round(2)}")

# Aceleração da variação da receita bruta anual
print(f"\nTotal da aceleração da variação anual da receita bruta entre 1999 e 2020: R$ {df_va_analise['aceleracao_variacao_anual_va'].sum().round(2)}")
print(f"Média móvel anual da aceleração da variação anual da receita bruta: R${(df_va_analise['aceleracao_variacao_anual_va'].sum()/df_va_analise.shape[0]).round(2)}")

# Porcentagem da variação da receita bruta anual
print(f"\nPorcentagem da variação anual da receita bruta entre 1999 e 2020: {df_va_analise['porcentagem_variacao_anual_va_%'].sum().round(2)}%")
print(f"Média percentual móvel anual da variação anual da receita bruta: {(df_va_analise['porcentagem_variacao_anual_va_%'].sum()/df_va_analise.shape[0]).round(2)}%")

# Porcentagem da aceleração da variação da receita bruta anual
print(f"\nPorcentagem da aceleração da variação anual da receita bruta entre 1999 e 2020: {df_va_analise['porcentagem_aceleracao_variacao_anual_va_%'].sum().round(2)}%")
print(f"Média percentual móvel anual da aceleração da variação anual da receita bruta: {(df_va_analise['porcentagem_aceleracao_variacao_anual_va_%'].sum()/df_va_analise.shape[0]).round(2)}%")

In [ ]:
# Variação da receita bruta anual da agropecuária
print(f"Total da variação anual da receita bruta da agropecuária entre 1999 e 2020: R$ {df_va_analise['variacao_anual_va_agropecuaria'].sum().round(2)}")
print(f"Média móvel da variação da agropecuária anual: R$ {(df_va_analise['variacao_anual_va_agropecuaria'].sum()/df_va_analise.shape[0]).round(2)}")

# Aceleração da variação da receita bruta anual da agropecuária 
print(f"\nTotal da aceleração da variação anual da receita bruta da agropecuária entre 1999 e 2020: R$ {df_va_analise['aceleracao_variacao_anual_va_agropecuaria'].sum().round(2)}")
print(f"Média móvel anual da aceleração da variação anual da receita bruta da agropecuária: R${(df_va_analise['aceleracao_variacao_anual_va_agropecuaria'].sum()/df_va_analise.shape[0]).round(2)}")

# Porcentagem da variação da receita bruta anual da agropecuária 
print(f"\nPorcentagem da variação anual da receita bruta da agropecuária entre 1999 e 2020: {df_va_analise['porcentagem_variacao_anual_va_agropecuaria_%'].sum().round(2)}%")
print(f"Média percentual móvel anual da variação anual da receita bruta da agropecuária: {(df_va_analise['porcentagem_variacao_anual_va_agropecuaria_%'].sum()/df_va_analise.shape[0]).round(2)}%")

# Porcentagem da aceleração da variação da receita bruta anual da agropecuária 
print(f"\nPorcentagem da aceleração da variação anual da receita bruta da agropecuária entre 1999 e 2020: {df_va_analise['porcentagem_aceleracao_variacao_anual_va_agropecuaria_%'].sum().round(2)}%")
print(f"Média percentual móvel anual da aceleração da variação anual da receita bruta da agropecuária: {(df_va_analise['porcentagem_aceleracao_variacao_anual_va_agropecuaria_%'].sum()/df_va_analise.shape[0]).round(2)}%")

In [ ]:
# Variação da receita bruta anual da indústria
print(f"Total da variação anual da receita bruta da indústria entre 1999 e 2020: R$ {df_va_analise['variacao_anual_va_industria'].sum().round(2)}")
print(f"Média móvel da variação da indústria anual: R$ {(df_va_analise['variacao_anual_va_industria'].sum()/df_va_analise.shape[0]).round(2)}")

# Aceleração da variação da receita bruta anual da indústria 
print(f"\nTotal da aceleração da variação anual da receita bruta da indústria entre 1999 e 2020: R$ {df_va_analise['aceleracao_variacao_anual_va_industria'].sum().round(2)}")
print(f"Média móvel anual da aceleração da variação anual da receita bruta da indústria: R${(df_va_analise['aceleracao_variacao_anual_va_industria'].sum()/df_va_analise.shape[0]).round(2)}")

# Porcentagem da variação da receita bruta anual da indústria 
print(f"\nPorcentagem da variação anual da receita bruta da indústria entre 1999 e 2020: {df_va_analise['porcentagem_variacao_anual_va_industria_%'].sum().round(2)}%")
print(f"Média percentual móvel anual da variação anual da receita bruta da indústria: {(df_va_analise['porcentagem_variacao_anual_va_industria_%'].sum()/df_va_analise.shape[0]).round(2)}%")

# Porcentagem da aceleração da variação da receita bruta anual da indústria 
print(f"\nPorcentagem da aceleração da variação anual da receita bruta da indústria entre 1999 e 2020: {df_va_analise['porcentagem_aceleracao_variacao_anual_va_industria_%'].sum().round(2)}%")
print(f"Média percentual móvel anual da aceleração da variação anual da receita bruta da indústria: {(df_va_analise['porcentagem_aceleracao_variacao_anual_va_industria_%'].sum()/df_va_analise.shape[0]).round(2)}%")

In [ ]:
# Variação da receita bruta anual da serviços
print(f"Total da variação anual da receita bruta da serviços entre 1999 e 2020: R$ {df_va_analise['variacao_anual_va_adespss'].sum().round(2)}")
print(f"Média móvel da variação da serviços anual: R$ {(df_va_analise['variacao_anual_va_adespss'].sum()/df_va_analise.shape[0]).round(2)}")

# Aceleração da variação da receita bruta anual da serviços 
print(f"\nTotal da aceleração da variação anual da receita bruta da serviços entre 1999 e 2020: R$ {df_va_analise['aceleracao_variacao_anual_va_adespss'].sum().round(2)}")
print(f"Média móvel anual da aceleração da variação anual da receita bruta da serviços: R${(df_va_analise['aceleracao_variacao_anual_va_adespss'].sum()/df_va_analise.shape[0]).round(2)}")

# Porcentagem da variação da receita bruta anual da serviços 
print(f"\nPorcentagem da variação anual da receita bruta da serviços entre 1999 e 2020: {df_va_analise['porcentagem_variacao_anual_va_adespss_%'].sum().round(2)}%")
print(f"Média percentual móvel anual da variação anual da receita bruta da serviços: {(df_va_analise['porcentagem_variacao_anual_va_adespss_%'].sum()/df_va_analise.shape[0]).round(2)}%")

# Porcentagem da aceleração da variação da receita bruta anual da serviços 
print(f"\nPorcentagem da aceleração da variação anual da receita bruta da serviços entre 1999 e 2020: {df_va_analise['porcentagem_aceleracao_variacao_anual_va_adespss_%'].sum().round(2)}%")
print(f"Média percentual móvel anual da aceleração da variação anual da receita bruta da serviços: {(df_va_analise['porcentagem_aceleracao_variacao_anual_va_adespss_%'].sum()/df_va_analise.shape[0]).round(2)}%")

In [ ]:
# Variação da receita bruta anual da serviços exclusivos da Adm. Pública
print(f"Total da variação anual da receita bruta da serviços exclusivos da Adm. Pública entre 1999 e 2020: R$ {df_va_analise['variacao_anual_va_servicos'].sum().round(2)}")
print(f"Média móvel da variação da serviços anual: R$ {(df_va_analise['variacao_anual_va_servicos'].sum()/df_va_analise.shape[0]).round(2)}")

# Aceleração da variação da receita bruta anual da serviços 
print(f"\nTotal da aceleração da variação anual da receita bruta da serviços exclusivos da Adm. Pública entre 1999 e 2020: R$ {df_va_analise['aceleracao_variacao_anual_va_servicos'].sum().round(2)}")
print(f"Média móvel anual da aceleração da variação anual da receita bruta da serviços: R${(df_va_analise['aceleracao_variacao_anual_va_servicos'].sum()/df_va_analise.shape[0]).round(2)}")

# Porcentagem da variação da receita bruta anual da serviços 
print(f"\nPorcentagem da variação anual da receita bruta da serviços exclusivos da Adm. Pública entre 1999 e 2020: {df_va_analise['porcentagem_variacao_anual_va_servicos_%'].sum().round(2)}%")
print(f"Média percentual móvel anual da variação anual da receita bruta da serviços: {(df_va_analise['porcentagem_variacao_anual_va_servicos_%'].sum()/df_va_analise.shape[0]).round(2)}%")

# Porcentagem da aceleração da variação da receita bruta anual da serviços 
print(f"\nPorcentagem da aceleração da variação anual da receita bruta da serviços exclusivos da Adm. Pública entre 1999 e 2020: {df_va_analise['porcentagem_aceleracao_variacao_anual_va_servicos_%'].sum().round(2)}%")
print(f"Média percentual móvel anual da aceleração da variação anual da receita bruta da serviços: {(df_va_analise['porcentagem_aceleracao_variacao_anual_va_servicos_%'].sum()/df_va_analise.shape[0]).round(2)}%")

#### Impostos líquidos

In [ ]:
df_impostos_analise = df_valor_pib_analise.copy()
df_impostos_analise.drop(columns=['pib','va', 'va_agropecuaria', 'va_industria', 'va_servicos', 'va_adespss'], inplace=True)

pd.options.display.float_format = '{:.2f}'.format
print(df_impostos_analise)

In [ ]:
# inferindo variação e acelaração anual dos impostos líquidos
df_impostos_analise['variacao_anual'] = df_impostos_analise['impostos_liquidos'].diff()
df_impostos_analise['porcentagem_variacao_anual_%'] = ((df_impostos_analise['variacao_anual'] / df_impostos_analise['impostos_liquidos'].shift(1)) * 100).round(2)
df_impostos_analise['aceleracao_variacao_anual'] = df_impostos_analise['variacao_anual'].diff()
df_impostos_analise['porcentagem_aceleracao_variacao_anual_%'] = ((df_impostos_analise['aceleracao_variacao_anual'] / df_impostos_analise['variacao_anual'].shift(1)) * 100).round(2)
df_impostos_analise

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_impostos_analise.index, df_impostos_analise['impostos_liquidos'], label='Valor Anual - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Impostos municipais anual líquido entre os anos de 1999 a 2020')
ax.xaxis.label.set_visible(False)  # esconde a legenda do eixo X

# Configura o eixo x para não mostrar nenhum rótulo
# plt.gca().set_xticklabels([])

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_impostos_analise.index, df_impostos_analise['variacao_anual'], label='Valor Anual - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Variação dos impostos municipais líquidos entre os anos de 1999 a 2020')
ax.xaxis.label.set_visible(False)  # esconde a legenda do eixo X

# Configura o eixo x para não mostrar nenhum rótulo
# plt.gca().set_xticklabels([])

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_impostos_analise.index, df_impostos_analise['aceleracao_variacao_anual'], label='Valor Anual - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Aceleração da variação dos impostos municipais anual líquido entre os anos de 1999 a 2020')
ax.xaxis.label.set_visible(False)  # esconde a legenda do eixo X

# Configura o eixo x para não mostrar nenhum rótulo
# plt.gca().set_xticklabels([])

# Exibe o gráfico
plt.show()

In [ ]:
# Total da variação dos impostos líquidos
print(f"Total da variação dos impostos líquidos municipais entre 1999 e 2020: R$ {df_impostos_analise['variacao_anual'].sum().round(2)}")
print(f"Média móvel anual da variação dos impostos líquidos municipais: R$ {(df_impostos_analise['variacao_anual'].sum()/df_impostos_analise.shape[0]).round(2)}")

# Total da aceleração da variação dos impostos líquidos
print(f"\nTotal da aceleração da variação dos impostos líquidos municipais entre 1999 e 2020: R$ {df_impostos_analise['aceleracao_variacao_anual'].sum().round(2)}")
print(f"Média móvel anual da aceleração da variação dos impostos líquidos municipais: R$ {(df_impostos_analise['aceleracao_variacao_anual'].sum()/df_impostos_analise.shape[0]).round(2)}")

# Porcentagem da variação dos impostos líquidos
print(f"\nPorcentagem da variação dos impostos líquidos municipais entre 1999 e 2020: {df_impostos_analise['porcentagem_variacao_anual_%'].sum().round(2)}%")
print(f"Média móvel percentual anual da variação dos impostos líquidos municipais: {(df_impostos_analise['porcentagem_variacao_anual_%'].sum()/df_impostos_analise.shape[0]).round(2)}%")

# Porcentagem da aceleração da variação dos impostos líquidos
print(f"\nPorcentagem da aceleração da variação dos impostos líquidos municipais entre 1999 e 2020: {df_impostos_analise['porcentagem_aceleracao_variacao_anual_%'].sum().round(2)}%")
print(f"Média móvel percentual anual da acelereção variação dos impostos líquidos municipais: {(df_impostos_analise['porcentagem_aceleracao_variacao_anual_%'].sum()/df_impostos_analise.shape[0]).round(2)}%")

### Time series das receitas

#### Por meses

In [ ]:
# verificando se há valores nulos
df_receitas.isna().value_counts()

In [ ]:
df_receitas.dtypes

In [ ]:
# criando df para analises dos dados
df_receitas_analise = df_receitas[['DATA','VALOR_ARRECADADO']]

# criando coluna mês-ano para agrupar os dados
df_receitas_analise['mes_ano'] = pd.to_datetime(df_receitas_analise['DATA'].dt.strftime("%m-%Y"))

# agrupando os dados no df e deletando coluna das datas
df_receitas_analise.drop(columns=('DATA'), inplace=True)
df_receitas_analise = df_receitas_analise.groupby('mes_ano').sum()
df_receitas_analise

In [ ]:
df_receitas_analise['variacao_mensal'] = df_receitas_analise['VALOR_ARRECADADO'].diff()
df_receitas_analise['porcentagem_variacao_mensal_%'] = ((df_receitas_analise['variacao_mensal'] / df_receitas_analise['VALOR_ARRECADADO'].shift(1)) * 100).round(2)
df_receitas_analise['aceleracao_variacao_mensal'] = df_receitas_analise['variacao_mensal'].diff()
df_receitas_analise['porcentagem_aceleracao_variacao_mensal_%'] = ((df_receitas_analise['aceleracao_variacao_mensal'] / df_receitas_analise['variacao_mensal'].shift(1)) * 100).round(2)
df_receitas_analise

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_receitas_analise.index, df_receitas_analise['VALOR_ARRECADADO'], label='Valor Mensal - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Receitas orçamentária mensais brutas entre os anos de 2013 a 2022')
ax.xaxis.label.set_visible(False)  # esconde a legenda do eixo X

# Configura o eixo x para não mostrar nenhum rótulo
# plt.gca().set_xticklabels([])

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_receitas_analise.index, df_receitas_analise['variacao_mensal'], label='Valor Mensal - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Variação das receitas orçamentária mensais brutas entre os anos de 2013 a 2022')
ax.xaxis.label.set_visible(False)  # esconde a legenda do eixo X

# Configura o eixo x para não mostrar nenhum rótulo
# plt.gca().set_xticklabels([])

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_receitas_analise.index, df_receitas_analise['aceleracao_variacao_mensal'], label='Valor Mensal - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Aceleração da variação das receitas orçamentária mensais brutas entre os anos de 2013 a 2022')
ax.xaxis.label.set_visible(False)  # esconde a legenda do eixo X

# Configura o eixo x para não mostrar nenhum rótulo
# plt.gca().set_xticklabels([])

# Exibe o gráfico
plt.show()

In [ ]:
print(f"Total da variação das receitas mensais: R$ {df_receitas_analise['variacao_mensal'].sum().round(2)}")
print(f"Média móvel mensal da variação das receitas: R$ {(df_receitas_analise['variacao_mensal'].sum()/df_receitas_analise.shape[0]).round(2)}")

In [ ]:
print(f"Total da aceleração da variação das receitas mensais: R$ {df_receitas_analise['aceleracao_variacao_mensal'].sum().round(2)}")
print(f"Média móvel mensal da aceleração da variação das receitas: R$ {(df_receitas_analise['aceleracao_variacao_mensal'].sum()/df_receitas_analise.shape[0]).round(2)}")

In [ ]:
print(f"Porcentagem da variação das receitas mensais: {df_receitas_analise['porcentagem_variacao_mensal_%'].sum().round(2)}%")
print(f"Média movel percentual mensal da variação das receitas: {(df_receitas_analise['porcentagem_variacao_mensal_%'].sum()/df_receitas_analise.shape[0]).round(2)}%")

In [ ]:
print(f"Porcentagem da aceleração da variação das receitas mensais: {df_receitas_analise['porcentagem_aceleracao_variacao_mensal_%'].sum().round(2)}%")
print(f"Média móvel percentual mensal da acelereção variação das receitas: {(df_receitas_analise['porcentagem_aceleracao_variacao_mensal_%'].sum()/df_receitas_analise.shape[0]).round(2)}%")

#### Por ano

In [ ]:
# criando df para analises dos dados
df_receitas_analise = df_receitas[['DATA','VALOR_ARRECADADO']]

# criando coluna mês-ano para agrupar os dados
df_receitas_analise['ano'] = pd.to_datetime(df_receitas_analise['DATA'].dt.strftime("%Y"))

# agrupando os dados no df e deletando coluna das datas
df_receitas_analise.drop(columns=('DATA'), inplace=True)
df_receitas_analise = df_receitas_analise.groupby('ano').sum().round(2)

# Mostrar os valores com 2 casas decimais
pd.options.display.float_format = '{:.2f}'.format
print(df_receitas_analise)

In [ ]:
df_receitas_analise['variacao_anual'] = df_receitas_analise['VALOR_ARRECADADO'].diff()
df_receitas_analise['porcentagem_variacao_anual_%'] = ((df_receitas_analise['variacao_anual'] / df_receitas_analise['VALOR_ARRECADADO'].shift(1)) * 100).round(2)
df_receitas_analise['aceleracao_variacao_anual'] = df_receitas_analise['variacao_anual'].diff()
df_receitas_analise['porcentagem_aceleracao_variacao_anual_%'] = ((df_receitas_analise['aceleracao_variacao_anual'] / df_receitas_analise['variacao_anual'].shift(1)) * 100).round(2)
df_receitas_analise

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_receitas_analise.index, df_receitas_analise['VALOR_ARRECADADO'], label='Valor Anual - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Receitas orçamentária anuais brutas entre os anos de 2013 a 2022')
ax.xaxis.label.set_visible(False)  # esconde a legenda do eixo X

# Configura o eixo x para não mostrar nenhum rótulo
# plt.gca().set_xticklabels([])

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_receitas_analise.index, df_receitas_analise['variacao_anual'], label='Valor Anual - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Variação das receitas orçamentária anuais brutas entre os anos de 2013 a 2022')
ax.xaxis.label.set_visible(False)  # esconde a legenda do eixo X

# Configura o eixo x para não mostrar nenhum rótulo
# plt.gca().set_xticklabels([])

# Exibe o gráfico
plt.show()

In [ ]:
# Plota o gráfico de linha com as séries temporais
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_receitas_analise.index, df_receitas_analise['aceleracao_variacao_anual'], label='Valor Anual - R$')

# Adiciona legenda e título ao gráfico
ax.legend()
ax.set_title('Aceleração da variação das receitas orçamentária anuais brutas entre os anos de 2013 a 2022')
ax.xaxis.label.set_visible(False)  # esconde a legenda do eixo X

# Configura o eixo x para não mostrar nenhum rótulo
# plt.gca().set_xticklabels([])

# Exibe o gráfico
plt.show()

In [ ]:
print(f"Total da variação das receitas anuais: R$ {df_receitas_analise['variacao_anual'].sum().round(2)}")
print(f"Média móvel anual da variação das receitas: R$ {(df_receitas_analise['variacao_anual'].sum()/df_receitas_analise.shape[0]).round(2)}")

In [ ]:
print(f"Total da aceleração da variação das receitas anuais: R$ {df_receitas_analise['aceleracao_variacao_anual'].sum().round(2)}")
print(f"Média móvel anual da aceleração da variação das receitas: R$ {(df_receitas_analise['aceleracao_variacao_anual'].sum()/df_receitas_analise.shape[0]).round(2)}")

In [ ]:
print(f"Porcentagem da variação das receitas anuais: {df_receitas_analise['porcentagem_variacao_anual_%'].sum().round(2)}%")
print(f"Média movel percentual anual da variação das receitas: {(df_receitas_analise['porcentagem_variacao_anual_%'].sum()/df_receitas_analise.shape[0]).round(2)}%")

In [ ]:
print(f"Porcentagem da aceleração da variação das receitas anuais: {df_receitas_analise['porcentagem_aceleracao_variacao_anual_%'].sum().round(2)}%")
print(f"Média móvel percentual anual da acelereção variação das receitas: {(df_receitas_analise['porcentagem_aceleracao_variacao_anual_%'].sum()/df_receitas_analise.shape[0]).round(2)}%")

## Análise regressiva linear

### Gerando dados para o PIB e População para os anos de 2021 e 2022

In [ ]:
df_dados_analise_linear_receitas = df_receitas_analise.drop(columns=['variacao_anual','porcentagem_variacao_anual_%','aceleracao_variacao_anual','porcentagem_aceleracao_variacao_anual_%'])

df_dados_analise_linear_receitas = pd.merge(df_dados_analise_linear_receitas,
                                           df_ibge_populacao_index_ano.drop(columns=['variacao_anual','porcentagem_variacao_anual_%','aceleracao_variacao_anual','porcentagem_aceleracao_variacao_anual_%']),
                                           left_index=True, right_index=True, how='inner')

df_dados_analise_linear_receitas = pd.merge(df_dados_analise_linear_receitas,
                                           df_valor_pib_analise,
                                           left_index=True, right_index=True, how='inner')


df_dados_analise_linear_receitas['pib_percapita'] = ((df_dados_analise_linear_receitas['pib']/df_dados_analise_linear_receitas['populacao'])*100).round(2)

anos = pd.date_range(start='2013-01-01', end='2020-01-01', freq='AS')
df_dados_analise_linear_receitas['anos'] = anos.year

df_dados_analise_linear_receitas

In [ ]:
df_dados_analise_linear_receitas.dtypes

In [ ]:
print('Matriz de correlação entre as variáveis:')
df_dados_analise_linear_receitas.corr()

In [ ]:
# plotando heatmap
sns.heatmap(df_dados_analise_linear_receitas.corr(), annot=True)

#### Análise regressiva linear

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np

##### Modelo para análise regressiva linear com o ANO como variável dependente

In [ ]:
df_receitas_analise

In [ ]:
# dados
X = df_dados_analise_linear_receitas[['anos']]
y = np.array(df_dados_analise_linear_receitas['VALOR_ARRECADADO']).reshape((-1,1))

# Divide o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# cria o modelo de regressão linear e ajusta os dados
model = LinearRegression().fit(X_train,y_train)

# utilizando o modelo para previsões nos dados de teste
y_pred = model.predict(X_test)

# calcula a previsão para um novo dado
ano = 2022
y_pred = model.predict([[ano]])

# Imprime os coeficientes da regressão
print(f'Coeficiente de interceptação: {model.intercept_[0]:.2f}')

# Avaliar o modelo usando os dados de teste
score = model.score(X_test, y_test)
print(f"A acurácia do modelo é {score:.2f}")

# Imprime a previsão para o novo dado
print(f'Previsão para o ano de {ano}: R$ {float(y_pred[0]):.2f}')

In [ ]:
valor_arrecadado = 201644040.94
diferenca = valor_arrecadado - y_pred[0][0]
porcentagem = (diferenca/valor_arrecadado).round(2)
print(f'O modelo possui um desvio da realidade de aproximadamente: {porcentagem}%.')

##### Modelo para análise regressiva linear com a POPULACAO como variável dependente

In [ ]:
df_ibge_populacao

In [ ]:
# dados
X = df_dados_analise_linear_receitas[['populacao']]
y = np.array(df_dados_analise_linear_receitas['VALOR_ARRECADADO']).reshape((-1,1))

# Divide o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# cria o modelo de regressão linear e ajusta os dados
model = LinearRegression().fit(X_train,y_train)

# utilizando o modelo para previsões nos dados de teste
y_pred = model.predict(X_test)

# calcula a previsão para um novo dado
populacao = 62576
y_pred = model.predict([[populacao]])

# Imprime os coeficientes da regressão
print(f'Coeficiente de interceptação: {model.intercept_[0]:.2f}')

# Avaliar o modelo usando os dados de teste
score = model.score(X_test, y_test)
print(f"A acurácia do modelo é {score:.2f}")

# Imprime a previsão para o novo dado
print(f'Previsão para a população de {populacao}: R$ {float(y_pred[0]):.2f}')

In [ ]:
valor_arrecadado = 179003057.61
diferenca = valor_arrecadado - y_pred[0][0]
porcentagem = (diferenca/valor_arrecadado).round(2)
print(f'O modelo possui um desvio da realidade de aproximadamente: {porcentagem}%.')

##### Modelo para análise regressiva linear com a POPULACAO e ANO como variáveis dependentes

In [ ]:
# dados
X = df_dados_analise_linear_receitas[['populacao','anos']]
y = np.array(df_dados_analise_linear_receitas['VALOR_ARRECADADO']).reshape((-1,1))

# Divide o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# cria o modelo de regressão linear e ajusta os dados
model = LinearRegression().fit(X_train,y_train)

# utilizando o modelo para previsões nos dados de teste
y_pred = model.predict(X_test)

# calcula a previsão para um novo dado
populacao = 62576
ano = 2021
y_pred = model.predict([[populacao,ano]])

# Imprime os coeficientes da regressão
print(f'Coeficiente de interceptação: {model.intercept_[0]:.2f}')

# Avaliar o modelo usando os dados de teste
score = model.score(X_test, y_test)
print(f"A acurácia do modelo é {score:.2f}")

# Imprime a previsão para o novo dado
print(f'Previsão para a população de {populacao} no ano de {ano}: R$ {float(y_pred[0]):.2f}')

In [ ]:
valor_arrecadado = 179003057.61
diferenca = valor_arrecadado - y_pred[0][0]
porcentagem = (diferenca/valor_arrecadado).round(2)
print(f'O modelo possui um desvio da realidade de aproximadamente: {porcentagem}%.')

#### Modelo para análise regressiva linear com o PIB, IMPOSTOS E VALORES BRUTOS como variáveis dependentes

In [ ]:
df_dados_analise_linear_receitas

In [ ]:
# dados
X = df_dados_analise_linear_receitas[['pib','pib_percapita','impostos_liquidos','va','va_agropecuaria','va_industria','va_servicos','va_adespss']]
y = np.array(df_dados_analise_linear_receitas['VALOR_ARRECADADO']).reshape((-1,1))

# Divide o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# cria o modelo de regressão linear e ajusta os dados
model = LinearRegression().fit(X_train,y_train)

# utilizando o modelo para previsões nos dados de teste
y_pred = model.predict(X_test)

# calcula a previsão para um novo dado
pib = 1234470600.00
pib_percapita = 1821301.67
imposts_liquidos = 149580220.00
va = 984890380.00
va_agropecuaria = 16844650.00
va_industria = 98240130.00
va_servicos = 567634700.00
va_adespss = 302170900.00

y_pred = model.predict([[pib,pib_percapita,imposts_liquidos,va,va_agropecuaria,va_industria,va_servicos,va_adespss]])

# Imprime os coeficientes da regressão
print(f'Coeficiente de interceptação: {model.intercept_[0]:.2f}')

# Avaliar o modelo usando os dados de teste
score = model.score(X_test, y_test)
print(f"A acurácia do modelo é {score:.2f}")

# Imprime a previsão para o novo dado
print(f'Previsão: R$ {float(y_pred[0]):.2f}')

##### Modelo para análise regressiva linear com o PIB, POPULACAO E ANOS como variáveis dependentes

In [ ]:
# dados
X = df_dados_analise_linear_receitas[['pib','populacao','anos']]
y = np.array(df_dados_analise_linear_receitas['VALOR_ARRECADADO']).reshape((-1,1))

# Divide o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# cria o modelo de regressão linear e ajusta os dados
model = LinearRegression().fit(X_train,y_train)

# utilizando o modelo para previsões nos dados de teste
y_pred = model.predict(X_test)

# calcula a previsão para um novo dado
pib = 1234470600.00
populacao = 62556
ano = 2023

y_pred = model.predict([[pib,populacao,ano]])

# Imprime os coeficientes da regressão
print(f'Coeficiente de interceptação: {model.intercept_[0]:.2f}')

# Avaliar o modelo usando os dados de teste
score = model.score(X_test, y_test)
print(f"A acurácia do modelo é {score:.2f}")

# Imprime a previsão para o novo dado
print(f'Previsão: R$ {float(y_pred[0]):.2f}')